In [60]:
#all imports for this workbook

import numpy as np
import pandas as pd
import time
import xport

In [37]:
#change these paths as necessary
data_path = '/Users/dhawan/Documents/K2/exploratory_analysis/health_project/data/rawLLCP2017.XPT'
hdf_path = '/Users/dhawan/Documents/K2/exploratory_analysis/health_project/data/interim/brfss.h5'

## Read Data from XPT to DataFrame

This section reads the 2017 xpt dataset from BRFSS into a DataFrame and then stores it in an HDF store for easy access and retrieval,

In [28]:
#read the xpt file into a pandas DataFrame

tic = time.time()
with open(data_path, 'rb') as f:
    df = xport.to_dataframe(f)
toc = time.time()
print("Sucessfully read into DataFrame in {} seconds".format(toc-tic))

In [30]:
#make sure everything was captured
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450016 entries, 0 to 450015
Columns: 358 entries, _STATE to _AIDTST3
dtypes: float64(352), object(6)
memory usage: 1.2+ GB


In [35]:
#pythonize the column names
df.columns = [x.lower() for x in df.columns]

In [38]:
# store the entire dataset in an HDF store
df.to_hdf(hdf_path, 'raw_17')

## Optimize Data Storage

To reduce the burden on memory, this section will extract the relevant columns we are interested in for our analysis, downcast numeric values where appropriate, and convert several of the columns into categorical variables for easy interpretation. First, let's load the data if we haven't already.

In [ ]:
# If data not yet loaded
%time df = pd.read_hdf(hdf_path, 'raw_17')

### Take Out Columns

The raw dataset is over 1 Gb. Let's see how much we can reduce this when we take out columns irrelevant to the project.

In [88]:
df.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450016 entries, 0 to 450015
Columns: 358 entries, _state to _aidtst3
dtypes: float64(352), object(6)
memory usage: 1.3 GB


The initial project question we created for the MVP was as follows:

>What is the state of substance use now (2017)? How does it differ across demographics and geography? Plot substance use across age, gender, education, salary, social determinants, chronic health conditions, and state

Thus, we will need to make sure to keep columns associated with basic user and demographic/geographic data as well as answers to certain sections of the questionnaire. Looking at the 2017 questionnaire, these include "Health Status", "Healthy Days", "Chronic Health Conditions", "Demographics", "Tobacco Use", "E-Cigarettes", "Alcohol Consumption", "Healthy Days (Symptoms)", "Marijuana", "Emotional Support and Life Satisfaction", and "Social Determinants of Health".

Looking at the 2017 codebook, we can grab the variable names that help define these sections, find their indices, and use that to create an array of columns to keep. All other columns will be dropped.

In [78]:
# Array of tuples specifying columns to keep (e.g., (start,end))
# These correspond to the relevant questionnarie sections, and 
# unfortunately will have to be manually generated for each years 
# data set given the variable names and questions may change
col_tuples = [(0,30), (40,52), (57,88), (158,161), (199,201), (228,236)]
col_nums_to_keep = [np.arange(x[0],x[1]+1) for x in col_tuples]
col_nums_to_keep = np.concatenate([x for x in col_nums_to_keep], axis=None) #unstack arrays
cols = pd.Series(df.columns[col_nums_to_keep]).values #convert to column names

In [80]:
#do a quick check to make sure it worked
cols[-10:]

array(['rsnmrjna', 'emtsuprt', 'lsatisfy', 'sdhbills', 'sdhmove',
       'howsafe1', 'sdhfood', 'sdhmeals', 'sdhmoney', 'sdhstres'],
      dtype=object)

In [82]:
#trim the DataFrame to relevant columns
df_trim = df.loc[:,cols]

In [87]:
df_trim.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450016 entries, 0 to 450015
Columns: 92 entries, _state to sdhstres
dtypes: float64(87), object(5)
memory usage: 432.2 MB


Great, we've reduced the number of columns to 92, and reduced the memory from >1 Gb to ~400 Mb.

### Optimize Data Types and Remove Duplicates

For the next series of optimization / cleaning steps, we'll take out any duplicate rows and then downcast several columns to categorical or small unsigned integer types.

In [89]:
df_trim.drop_duplicates(inplace=True)

In [91]:
df_trim.shape

(450016, 92)

Looks like there weren't any duplicates. Oh well...

Let's use the pandas profiling package to take a quick peek at the variables.

In [ ]:
def mem_usage(df):
    #Print the memory usage
    for dtype in ['float','int','object']:
        selected_dtype = df_trim.select_dtypes(include=[dtype])
        mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
        mean_usage_mb = mean_usage_b / 1024 ** 2
        print("Average memory usage for {} columns: {:03.2f}MB".format(dtype,mean_usage_mb))

In [103]:
mem_usage(df_trim)

Average memory usage for float columns: 3.66MB
Average memory usage for int columns: 23.43MB
Average memory usage for object columns: 26.09MB


We can see that the object columns are taking a lot of memory. Looking at these, we can probably still drop more of them even though we dropped a lot in the last iteration.

In [98]:
df_trim.select_dtypes(include='object').columns

Index(['idate', 'imonth', 'iday', 'iyear', 'seqno'], dtype='object')

In [112]:
df_trim.drop(['idate', 'imonth', 'iday', 'iyear'], axis=1, inplace=True)

Now, let's turn to the numeric variables. A quick look at the columns makes it clear that they don't need to be float. There are no decimal points other than 0, and in most cases, the values fall within a defined range based on the coding document. We'll just downcast all the numeric variables to an int.

In [110]:
#let's take a look
pd.options.display.max_columns=100
df_trim.select_dtypes(include='float').head()

,_state,fmonth,dispcode,_psu,ctelenm1,pvtresd1,colghous,statere1,cellfon4,ladult,numadult,nummen,numwomen,safetime,ctelnum1,cellfon5,cadult,pvtresd3,cclghous,cstate1,landline,hhadult,genhlth,physhlth,menthlth,poorhlth,cvdinfr4,cvdcrhd4,cvdstrk3,asthma3,asthnow,chcscncr,chcocncr,chccopd1,havarth3,addepev2,chckidny,diabete3,diabage2,sex,marital,educa,renthom1,numhhol2,numphon2,cpdemo1a,veteran3,employ1,children,income2,internet,weight2,height3,pregnant,deaf,blind,decide,diffwalk,diffdres,diffalon,smoke100,smokday2,stopsmk2,lastsmk2,usenow3,ecigaret,ecignow,alcday5,avedrnk2,drnk3ge5,maxdrnks,painact2,qlmentl2,qlstres2,qlhlth2,marijana,usemrjn1,rsnmrjna,emtsuprt,lsatisfy,sdhbills,sdhmove,howsafe1,sdhfood,sdhmeals,sdhmoney,sdhstres
0,1.0,1.0,1100.0,2.017000e+09,1.0,1.0,NaN,1.0,2.0,NaN,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,88.0,88.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,1.0,2.0,2.0,1.0,68.0,2.0,3.0,6.0,1.0,2.0,NaN,1.0,1.0,7.0,88.0,6.0,1.0,162.0,505.0,NaN,1.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,NaN,NaN,3.0,2.0,NaN,888.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,1100.0,2.017000e+09,1.0,1.0,NaN,1.0,2.0,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,88.0,88.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,3.0,NaN,1.0,1.0,6.0,1.0,1.0,2.0,1.0,2.0,7.0,88.0,8.0,1.0,211.0,511.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,3.0,2.0,NaN,888.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1100.0,2.017000e+09,1.0,1.0,NaN,1.0,2.0,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,88.0,88.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,3.0,NaN,1.0,1.0,2.0,1.0,2.0,NaN,1.0,2.0,7.0,88.0,99.0,2.0,195.0,602.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,3.0,2.0,NaN,101.0,1.0,88.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1200.0,2.017000e+09,1.0,1.0,NaN,1.0,2.0,NaN,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,88.0,88.0,NaN,2.0,2.0,2.0,2.0,NaN,1.0,2.0,1.0,2.0,1.0,2.0,1.0,73.0,2.0,3.0,3.0,1.0,2.0,NaN,1.0,2.0,7.0,88.0,1.0,2.0,170.0,507.0,NaN,2.0,2.0,1.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,3.0,2.0,NaN,888.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,1100.0,2.017000e+09,1.0,1.0,NaN,1.0,2.0,NaN,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,14.0,88.0,14.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,NaN,2.0,3.0,2.0,2.0,2.0,NaN,2.0,2.0,8.0,88.0,2.0,2.0,140.0,505.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,NaN,1.0,3.0,2.0,NaN,888.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#take out _psu column, it's redundant with seqno and it's a large value
df_trim.drop(['_psu'], axis=1, inplace=True)

In [126]:
#fill nan with -1 to facilitate conversion to int (-1 is never used in coding system)
#downcast all numeric variables to unsigned int
df_trim_numeric = df_trim.fillna(-1)\
                         .select_dtypes(include=['int','float'])\
                         .astype('int')\
                         .apply(pd.to_numeric, downcast='integer')

#merge into new df
df_trim_opt = df_trim.copy()
df_trim_opt[df_trim_numeric.columns] = df_trim_numeric

In [128]:
df_trim_opt.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450016 entries, 0 to 450015
Columns: 87 entries, _state to sdhstres
dtypes: int16(4), int8(82), object(1)
memory usage: 70.8 MB


We significantly reduced the size of the optimized dataset to 70 MB. Let's just check to make sure everything worked as expected.

In [131]:
df_trim_opt.head()

,_state,fmonth,dispcode,seqno,ctelenm1,pvtresd1,colghous,statere1,cellfon4,ladult,numadult,nummen,numwomen,safetime,ctelnum1,cellfon5,cadult,pvtresd3,cclghous,cstate1,landline,hhadult,genhlth,physhlth,menthlth,poorhlth,cvdinfr4,cvdcrhd4,cvdstrk3,asthma3,asthnow,chcscncr,chcocncr,chccopd1,havarth3,addepev2,chckidny,diabete3,diabage2,sex,marital,educa,renthom1,numhhol2,numphon2,cpdemo1a,veteran3,employ1,children,income2,internet,weight2,height3,pregnant,deaf,blind,decide,diffwalk,diffdres,diffalon,smoke100,smokday2,stopsmk2,lastsmk2,usenow3,ecigaret,ecignow,alcday5,avedrnk2,drnk3ge5,maxdrnks,painact2,qlmentl2,qlstres2,qlhlth2,marijana,usemrjn1,rsnmrjna,emtsuprt,lsatisfy,sdhbills,sdhmove,howsafe1,sdhfood,sdhmeals,sdhmoney,sdhstres
0,1,1,1100,2017000001,1,1,-1,1,2,-1,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,88,88,-1,2,2,2,2,-1,2,2,2,1,2,2,1,68,2,3,6,1,2,-1,1,1,7,88,6,1,162,505,-1,1,2,2,1,2,2,2,-1,-1,-1,3,2,-1,888,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1,1,1100,2017000002,1,1,-1,1,2,-1,2,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,88,88,-1,2,2,2,2,-1,2,2,2,2,2,2,3,-1,1,1,6,1,1,2,1,2,7,88,8,1,211,511,-1,2,2,2,2,2,2,2,-1,-1,-1,3,2,-1,888,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1,1,1100,2017000003,1,1,-1,1,2,-1,2,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,88,88,-1,2,2,2,2,-1,2,2,2,2,2,2,3,-1,1,1,2,1,2,-1,1,2,7,88,99,2,195,602,-1,2,2,2,2,2,2,2,-1,-1,-1,3,2,-1,101,1,88,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1,1,1200,2017000004,1,1,-1,1,2,-1,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,88,88,-1,2,2,2,2,-1,1,2,1,2,1,2,1,73,2,3,3,1,2,-1,1,2,7,88,1,2,170,507,-1,2,2,1,2,2,2,2,-1,-1,-1,3,2,-1,888,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1,1,1100,2017000005,1,1,-1,1,2,-1,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,14,88,14,2,2,2,1,1,2,2,1,2,2,2,3,-1,2,3,2,2,2,-1,2,2,8,88,2,2,140,505,-1,2,2,2,2,2,2,1,3,-1,1,3,2,-1,888,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


All looks good, we'll save this trimmed and datatype optimized dataset. There could likely be more optimization (e.g., converting to categoricals), but we'll leave that for now and move quickly to exploring the data. But first, coffee :)

In [130]:
# store the trimmed and datatype optimized dataset in an HDF store
df_trim_opt.to_hdf(hdf_path, 'trim_17')